# Visualizing Embeddings with TSNE

This is a demo to see the embeddings generated 

In [3]:
import torch as t
import transformers as trfl
from datasets import load_dataset
import matplotlib.pyplot as plt

## Data

We will be using the news dataset (okite97/news-data) from HuggingFace.


In [5]:
dataset = load_dataset("okite97/news-data")

Generating test split: 100%|██████████| 828/828 [00:00<00:00, 159255.46 examples/s]
